In [1]:
import pandas as pd
import urllib.request
from gensim.models.fasttext import FastText
from konlpy.tag import Okt
import numpy as np

In [2]:
train_data = pd.read_csv("data.csv", header=None)
label_data = pd.read_csv("data2.csv", header=None)

In [5]:
#결측값 제거
train_data = train_data.dropna(how = 'any')

#필요없는 단일문자 제거
for i in range(len(train_data)):
    train_data.iloc[i] = train_data.iloc[i].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

False


In [8]:
#학습이 가능하도록 데이터 정리
new_train = []
for i in train_data:
    if i == 0:
        continue
    for j in train_data[i]:
        new_train.append(j)

train = pd.DataFrame(new_train)

In [9]:
#불용어 설정
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','을','를','으로','자','에','와','한','하다']

In [11]:
#Okt 패키지를 이용해서 각 문장을 토큰화하고 불용어를 제거
okt = Okt()
tokenized_data = []
for sentence in train[0]:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

In [12]:
#벡터값의 비교를 위해서 gensim의 fastText를 이용해서 학습해두기
model = FastText(tokenized_data, vector_size=300, window=5, min_count=5, workers=4, sg=1)

In [13]:
#단어들에 대해 유사어를 정렬하기 위해서 FastText에서 벡터화된 단어들 뽑아내기
vocab = list(model.wv.index_to_key)

#단어들 벡터 뽑아내기
X = model.wv[vocab]

In [14]:
#길이가 1인 단어들 제거하기
for j in range(10):
    for i in vocab:
        length = len(i.replace(' ', ''))
        if(length == 1):
            vocab.remove(i)

In [16]:
#각 단어들의 빈도수 체크하기
vocab_count = []

for i in range(len(vocab)):
    cnt = 0 
    for j in tokenized_data:
        cnt += j.count(vocab[i])
    vocab_count.append(cnt)

#상위 100개 단어들 뽑기
top_vocab = vocab[:100]
len(top_vocab)

100

In [17]:
#fastText에서 빈도수 top100에 대한 유사어 정리
result_fastText = {}
for i in top_vocab:
    sim = []
    for j in range(10):
        sim.append(model.wv.most_similar(i)[j][0])
    result_fastText[i] = sim

In [19]:
#결과물
result_fastText

{'있다': ['생기', '깨지다', '발생', '중요하다', '이라는', '쌓이다', '부족하다', '전화하다', '인하다', '볼'],
 '않다': ['상대', '맞다', '설득', '유리하다', '고통', '다르다', '원하다', '하지만', '상대방', '피해'],
 '생각': ['웬만하다', '만약', '나르다', '산만하다', '주장', '나쁘다', '이야기', '왠만하다', '마음', '잘해주다'],
 '사람': ['다른', '믿다', '주변', '다르다', '고통', '굽히다', '라고', '인정', '공감', '사귀다'],
 '같다': ['없다', '적다', '딱하다', '건', '그렇다', '구분', '느낌', '아무', '막', '상상'],
 '되다': ['인하다', '도움', '아무', '잘못', '이유', '어쩔', '당하다', '어렵다', '딱하다', '무엇'],
 '없다': ['적다', '딱하다', '아무', '때문', '당연하다', '어쩔', '경험', '전혀', '혼동', '당하다'],
 '보다': ['들어가다', '어리다', '글', '즐기다', '잠', '장문', '써다', '아침', '월', '카페'],
 '그렇다': ['그렇게', '아니다', '막', '그렇다고', '별로', '라는', '딱하다', '그런', '이렇게', '싫다'],
 '하고': ['대화', '사과', '통해', '후', '해결', '풀다', '불구', '기다', '다툼', '다음'],
 '에서': ['부터', '대학교', '고등학교', '대학', '중학교', '초등학교', '팀', '반', '동료', '졸업'],
 '들다': ['들어서다', '어떻다', '즉흥', '그림', '기분', '구체', '감', '듯', '거들다', '인가요'],
 '친구': ['중학교', '두', '친하다', '고등학교', '지내다', '다니다', '랑', '처음', '학교', '언니'],
 '아니다': ['그렇게', '막', '그렇다', '그렇다고', '그런', '정말', '

In [21]:
#Bert 모델 불러오고 pretrained 모델 가져오기
from kobert_tokenizer import KoBERTTokenizer
import torch
from transformers import BertModel

model_bert = BertModel.from_pretrained('skt/kobert-base-v1')
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [22]:
#기존에 토큰화된 데이터 옯기고
tokenized_data_bert = tokenized_data

#각 데이터 앞뒤에 시작, 종료를 나타내는 심볼 추가하기
for i in tokenized_data_bert:
    i.insert(0, '[CLS]')
    i.append('[SEP]')

In [24]:
#한글자까지 전부 포함해서 뽑고
#두글자 top100에서 유사한 단어 뽑아보기 -> vocab 재정의 필요 -> fastText에서도 한글자는 이용되긴 함
vocab_new = list(model.wv.index_to_key)     
result_bert = {}
for i in vocab_new:
    result_bert[i] = torch.FloatTensor([0] * 768)
    
#갯수용 딕셔너리
result_cnt = {}
for i in vocab_new:
    result_cnt[i] = 0

for sentence in tokenized_data_bert:
    indexed_text = torch.tensor([tokenizer.convert_tokens_to_ids(sentence)])
    segments_ids = torch.tensor([[1] * len(sentence)])
    
    model_bert.eval()
    
    encoded_layers = model_bert(indexed_text, segments_ids)
    embedded_layer = encoded_layers.last_hidden_state.squeeze()
    
    #각 단어에 해당하는 딕셔너리에서 값 갱신 필요 -> 평균을 구해야함 not 갱신
    #총 몇개로 나누어야하는지가 문제 -> 갯수용 딕셔너리를 추가하고 값을 더하는 방식으로 수정
    #결과 다 끝난 후 갯수로 나누어주기
    #vocab에서 사라지는 단어들도 있는 이슈 발생 -> 1. tokenized 합치고 unique로 vocab뽑기 2. 의미없는거니까 그냥 무시하기
    #2번으로 일단 해보기
    for i, ch in enumerate(sentence):
        if ch in vocab_new:
            if ch == '[CLS]' or ch == '[SEP]': continue
            result_bert[ch] += embedded_layer[i].detach()
            result_cnt[ch] += 1

for i in result_bert:
    result_bert[i] = result_bert[i] / result_cnt[i]

In [25]:
for i in result_bert:
    print(i, result_bert[i][:5])

있다 tensor([ 0.0077,  0.0510,  0.0230, -4.6489,  0.0991])
것 tensor([ 0.1466, -0.0625,  0.3126, -4.1251,  0.1827])
않다 tensor([ 0.0991, -0.1229,  0.2912, -4.9780,  0.1957])
생각 tensor([ 0.0593, -0.1169,  0.2582, -5.0332,  0.2160])
적 tensor([ 0.1664, -0.2066,  0.4020, -4.3611,  0.0487])
사람 tensor([-0.0330,  0.0664,  0.0989, -4.8229,  0.0231])
때 tensor([ 0.1537, -0.1033,  0.0459, -4.5065,  0.1500])
같다 tensor([ 0.0117, -0.1559,  0.2799, -4.9074,  0.2604])
되다 tensor([ 0.0870, -0.1323,  0.3060, -4.9139,  0.1916])
그 tensor([ 0.0978, -0.0975,  0.1517, -4.6501,  0.4160])
없다 tensor([ 0.0533, -0.1137,  0.2765, -5.0644,  0.1935])
내 tensor([ 0.0549,  0.0292,  0.0420, -4.5704,  0.2175])
보다 tensor([ 0.2768, -0.0892,  0.2543, -4.4140,  0.2092])
저 tensor([ 0.0093,  0.0546,  0.0456, -4.8837,  0.2115])
나 tensor([ 0.1177, -0.0903,  0.1574, -4.9111,  0.2580])
그렇다 tensor([ 0.0520, -0.1780,  0.2774, -4.6858,  0.1809])
다 tensor([ 0.0678, -0.0346,  0.2329, -5.0888,  0.2609])
일 tensor([ 0.1017,  0.0231,  0.2367, -

In [26]:
#1. 버트에 맞는 토크나이저를 사용하지 않았기에 기대하는 성능이 나오지 않을 수 있음
#2. 유사도 측정방식에 따른 결과 변동도 있을 수 있음
#3. 학습시 한글자를 빼고 생각하기는 어려움(fasttext도 일단 포함해서 나옴)
#4. 벡터를 낸 후 한글자를 제외하고 유사어를 찾아보기

del_vocab = []
for i in result_bert:
    if len(i) == 1:
        del_vocab.append(i)

for i in del_vocab:
    del result_bert[i]

sim_bert = {}
sim_bert_vocab = {}
for i in top_vocab:
    sim_bert[i] = []
    sim_bert_vocab[i] = []

In [27]:
#모든 단어들에 대해서 유사도 정리하기
from scipy.spatial.distance import cosine

for i in sim_bert:
    same_ = []
    same_vocab = []
    for j in result_bert:
        if i == j:
            continue
        same_.append(1 - cosine(result_bert[i], result_bert[j]))
        same_vocab.append(j)
    sim_bert[i] = same_
    sim_bert_vocab[i] = same_vocab
    
for i in sim_bert:
    for j in range(len(sim_bert[i])):
        for k in range(j, len(sim_bert[i])):
            
            if sim_bert[i][j] < sim_bert[i][k]:
                tmp = sim_bert[i][j]
                sim_bert[i][j] = sim_bert[i][k]
                sim_bert[i][k] = tmp
                
                tmp = sim_bert_vocab[i][j]
                sim_bert_vocab[i][j] = sim_bert_vocab[i][k]
                sim_bert_vocab[i][k] = tmp

In [28]:
for i in sim_bert_vocab:
    print(i, ":", sim_bert_vocab[i][:10])

있다 : ['없이', '만큼', '처음', '지금', '진짜', '이지만', '최근', '밖에', '아직', '스럽다']
않다 : ['되다', '나다', '이야기', '많다', '주다', '싶다', '해주다', '맞추다', '생각', '자다']
생각 : ['없다', '행동', '되다', '이야기', '나다', '싶다', '많다', '좋다', '가다', '아니다']
사람 : ['지금', '만큼', '이상', '처음', '이지만', '공간', '마음', '이라도', '인지', '이라는']
같다 : ['좋아하다', '선호', '좋다', '없다', '아니다', '많다', '생각', '힘들다', '들다', '편하다']
되다 : ['않다', '많다', '해주다', '생각', '싶다', '나다', '주다', '받다', '어렵다', '되어다']
없다 : ['행동', '생각', '많다', '아니다', '좋다', '되다', '싶다', '가다', '들다', '이야기']
보다 : ['없이', '보다는', '만큼', '하지만', '결과', '처럼', '관련', '에는', '진짜', '이라도']
그렇다 : ['어떻다', '그냥', '얘기', '좋다', '살다', '알다', '힘들다', '많다', '나오다', '싶다']
하고 : ['보고', '하지만', '이라고', '치고', '보다는', '스럽다', '라고', '에는', '에서는', '에서도']
에서 : ['에서는', '에서도', '없이', '보다는', '하지만', '에는', '스럽게', '함께', '위로', '이지만']
들다 : ['많다', '없다', '생각', '좋다', '행동', '싶다', '가다', '되다', '어떻다', '싫다']
친구 : ['얘기', '그렇다', '부분', '좋다', '알다', '어떻다', '그냥', '그렇게', '오다', '뭔가']
아니다 : ['없다', '많이', '자다', '맞다', '생각', '가다', '어떻다', '행동', '많다', '항상']
시간 : ['상황', '기간', '과정', '방식', '

In [33]:
while True:
    control = int(input("원하는 명령을 입력하세요(1. 검색, 2. 종료) : "))
    
    if control == 1:
        key = input("유사어를 찾고싶은 단어를 입력하세요. : ")
        if not key in sim_bert_vocab:
            print(key, "는 존재하지 않는 단어입니다!")
            continue
        
        rank = int(input("원하는 출력 갯수를 입력하세요. : "))
            
        c_type = int(input("원하는 유사어 종류를 입력하세요(1. 전부, 2. 체언, 3. 용언) : "))
        if c_type == 1:
            print("기준 단어 :", key)
            cnt = 0
            for i in range(len(sim_bert_vocab[key])):
                print(cnt + 1, "순위 :", sim_bert_vocab[key][i], "/ 유사도 :", sim_bert[key][i])
                cnt += 1
                if cnt == rank:
                    break
                
            
        elif c_type == 2:
            print("기준 단어 :", key)
            cnt = 0
            for i in range(len(sim_bert_vocab[key])):
                check = True if okt.pos(sim_bert_vocab[key][i])[0][1] == 'Noun' else False
                
                if check:
                    print(cnt + 1, "순위 :", sim_bert_vocab[key][i], "/ 유사도 :", sim_bert[key][i])
                    cnt += 1
                    if cnt == rank:
                        break
                else:
                    continue
            
        else:
            print("기준 단어 :", key)
            cnt = 0
            for i in range(len(sim_bert_vocab[key])):
                check = True if okt.pos(sim_bert_vocab[key][i])[0][1] == 'Adjective' else False
                
                if check:
                    print(cnt + 1, "순위 :", sim_bert_vocab[key][i], "/ 유사도 :", sim_bert[key][i])
                    cnt += 1
                    if cnt == rank:
                        break
                else:
                    continue
                
        print()
    
    else:
        print("종료되었습니다!")
        break

기준 단어 : 친구
1 순위 : 얘기 / 유사도 : 0.9940896034240723
2 순위 : 부분 / 유사도 : 0.9937894940376282
3 순위 : 알다 / 유사도 : 0.9935139417648315
4 순위 : 그냥 / 유사도 : 0.993165135383606
5 순위 : 뭔가 / 유사도 : 0.992441713809967
6 순위 : 그때 / 유사도 : 0.9923856854438782
7 순위 : 입장 / 유사도 : 0.9923111200332642
8 순위 : 고민 / 유사도 : 0.9921601414680481
9 순위 : 이유 / 유사도 : 0.9920479655265808
10 순위 : 성격 / 유사도 : 0.9918030500411987

존재하지 않는 단어입니다!
기준 단어 : 관계
1 순위 : 결과 / 유사도 : 0.9363425970077515
2 순위 : 문제 / 유사도 : 0.9350759983062744
3 순위 : 거리 / 유사도 : 0.9295734763145447
4 순위 : 피해 / 유사도 : 0.925989031791687
5 순위 : 행사 / 유사도 : 0.925125241279602
6 순위 : 방향 / 유사도 : 0.9247291684150696
7 순위 : 상황 / 유사도 : 0.9241576194763184
8 순위 : 인사 / 유사도 : 0.9237880706787109
9 순위 : 이익 / 유사도 : 0.9236040115356445
10 순위 : 상태 / 유사도 : 0.9223005771636963

종료되었습니다!
